In [1]:
import pdfplumber
import nltk 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 
from nltk.tokenize import sent_tokenize
import faiss
from sentence_transformers import SentenceTransformer

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
import torch
import time
import re 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Snowwolf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Snowwolf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Snowwolf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Search through Tf-IDF

In [2]:
pdf_path = "book1.pdf"

pages = []
pdf = pdfplumber.open(pdf_path)

for i in tqdm(pdf.pages):
    text = i.extract_text() or ""
    pages.append(text)

paragraphs = [] 

for page in pages:
    paras = re.split(r"\n\s*\n", page) #Assuming every paragraph has 2 newline spaces separating it
    for i in paras:
        i.strip()
        paragraphs.append(i)

("Number of paragraphs are ",len(paragraphs))


100%|██████████| 228/228 [00:16<00:00, 14.21it/s]


('Number of paragraphs are ', 228)

In [3]:
from nltk.corpus import stopwords 
stop_words = stopwords.words("english")

vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    max_df=0.90, #Removes overly common (non-discriminative terrms)
    min_df=2 # Removes extremely rare terms (noise)
)

tfidf_matrix = vectorizer.fit_transform(paragraphs)
print("Tfidf Matrix shape is ", tfidf_matrix.shape)

Tfidf Matrix shape is  (228, 3150)


* Sparse Matrix of 228 rows (paragraphs) and 3150 feature matrix created for every paragraph ❔

In [4]:
query = "Lord Voldermort"
top_k = 4 

query_vector = vectorizer.transform([query]) #Generates a vec embedding of same shape as the tdidf one 
scores = cosine_similarity(query_vector, tfidf_matrix)[0]

idx = np.argsort(scores)[::-1][:top_k] #argsort returns the indexes of the elements according to their values (score) in ascending order and reverse -> top 5

for i in idx:
    print(f"Paragraph : {paragraphs[i][0:200]} \nScores : {scores[i]}\n")

Paragraph : When a letter arrives for unhappy but
ordinary Harry Potter, a decade-old secret
is revealed to him. His parents were
wizards, killed by a Dark Lord’s curse
when Harry was just a baby, and which he
so 
Scores : 0.12645905029115623

Paragraph : 210 Harry Potter
Hallowe’en like that, for all I knew you’d seen me coming to look
at what was guarding the Stone.’
‘You let the troll in?’
‘Certainly. I have a special gift with trolls – you must hav 
Scores : 0.08098958607063725

Paragraph : 54 Harry Potter
out, Harry wouldn’t have noticed it was there. The people hurry-
ing by didn’t glance at it. Their eyes slid from the big book shop
on one side to the record shop on the other as if th 
Scores : 0.08085259435317048

Paragraph : The Man with Two Faces 211
instructions – he is a great wizard and I am weak –’
‘You mean he was there in the classroom with you?’ Harry
gasped.
‘He is with me wherever I go,’ said Quirrell quietly. ‘ 
Scores : 0.07750553802544466



* Need to compute the *precision@k*, the ground truth required is not present with me currently so keeping it on hold 📓

# Embedding Similarity Search

In [5]:
pdf_path = "book1.pdf"

pages = []
pdf = pdfplumber.open(pdf_path)

for i in tqdm(pdf.pages):
    text = i.extract_text() or ""
    pages.append(text)
print("Number of pages are ",len(pages))

100%|██████████| 228/228 [00:16<00:00, 14.19it/s]

Number of pages are  228


In [6]:
sents_per_chunk = 6 
sent_overlap = 2

all_sentences = []
page_of_sentence = [] 

for page_index, page in enumerate(pages, start=1):
    for s in sent_tokenize(page):
        s = s.strip()
        all_sentences.append(s)
        page_of_sentence.append(page_index)

print("Total sentences ", len(all_sentences))

#Chunks with overlap
chunks = []
chunk_meta = []

i = 0
while i < len(all_sentences):
    start = i 
    end = min(i + sents_per_chunk, len(all_sentences))
    chunk_text = " ".join(all_sentences[start:end])
    chunks.append(chunk_text)
    chunk_meta.append({
        "start_sent" : start,
        "end_sent" : end-1,
        "start_page":page_of_sentence[start],
        "end_page": page_of_sentence[end-1]
    })
    i += sents_per_chunk - sent_overlap

print(f"Built {len(chunks)} chunks")

Total sentences  5085
Built 1272 chunks


In [7]:
model_id = "KaLM-Embedding/KaLM-embedding-multilingual-mini-instruct-v2.5"
model_arguments = {"dtype":torch.bfloat16,
                   "device_map":"cuda"
                   #"attn_implementation":"flash_attention_2"
                   }

model = SentenceTransformer(model_id,
                            cache_folder = "models",
                            model_kwargs = model_arguments)

In [8]:
#Batch speeds encoding and measure embedding time for profiling

batch = 16 
num_chunks = len(chunks)
embeddings = np.zeros((num_chunks, model.get_sentence_embedding_dimension()), dtype="float32") #There are 1272 chunks and this model's embedding is 1,896 -> (1272,896)

t0 = time.perf_counter() 
for i in tqdm(range(0, num_chunks, batch)):
    batch_texts = chunks[i:i+batch]
    embs = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False, batch_size=batch)
    embeddings[i:i+len(embs), :] = embs 
t1 = time.perf_counter()

print(f"Total {num_chunks} chunks embedded in {t1-t0:.3f}s \n Average {(t1-t0)/num_chunks:.4f}s per chunk")


100%|██████████| 80/80 [00:12<00:00,  6.49it/s]

Total 1272 chunks embedded in 12.329s 
 Average 0.0097s per chunk


In [9]:
norms = np.linalg.norm(embs, keepdims=True, axis=1)
norms[norms==0] = 1.0
embeddings_norm = embs/norms 
embeddings_norm = embeddings_norm.astype("float32")

# Langchain flow alternative

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.schema import Document

import json, os

In [11]:
pdf_path = "book1.pdf"
pdf = PyPDFLoader(pdf_path)
docs = pdf.load() 
print("Number of loaded documents are ", len(docs))
print("Example of one document \n ", docs[35].page_content[:250])

Number of loaded documents are  228
Example of one document 
  34 Harry Potter  
 
been trying to do. He shouted at Harry for about half an hour and 
then told him to go and make a cup of tea. Harry shuffled miser-
ably off into the kitchen, and by the time he got back, the post  
had arrived, right into Uncle V


In [12]:
chunk_size = 600 
chunk_overlap = 150 

splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                          chunk_overlap=chunk_overlap,
                                          separators=["\n\n", "\n", " ", ""]
                                          )
chunked_docs = splitter.split_documents(docs)
print("Chunks created ", len(chunked_docs))


Chunks created  1057


In [13]:
embeddings_model = HuggingFaceEmbeddings(model_name = "KaLM-Embedding/KaLM-embedding-multilingual-mini-instruct-v2.5",
                                         model_kwargs = {"device" : "cuda"},
                                         encode_kwargs = {"normalize_embeddings" : True})

#normalize cosine similarity = dot product of normalized vectors 
# Better ranking stability

* Each chunk -> Embedding vector
* Stored in FAISS Index 
* Metadata preserved

In [14]:
#vectorstore = FAISS.from_documents(chunked_docs, embedding=embeddings_model)
#vectorstore.save_local("faiss_book1_index")

In [15]:
#TO load the faiss vectorstore 
vectorstore = FAISS.load_local("faiss_book1_index",
                               embeddings=embeddings_model,
                               allow_dangerous_deserialization=True)

In [16]:
# Semantic (Similarity) Search 

query = "What was the color of Harry's eyes ?"

docs = vectorstore.similarity_search(query=query,
                                     k=3)

for i, d in enumerate(docs):
    print(f"\nResult {i+1}")
    print(d.page_content[0:100])
    print("Metadata : ", d.metadata)


Result 1
They just looked at him, sm iling. And slowly, Harry looked 
into the faces of the other people in t
Metadata :  {'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2012-05-01T23:57:27+10:00', 'subject': "When a letter arrives for unhappy but ordinary Harry Potter, a decade-old secret is revealed to him. His parents were wizards, killed by a Dark Lord's curse when Harry was just a baby, and which he somehow survived. Escaping from his unbearable Muggle guardians to Hogwarts, a wizarding school brimming with ghosts and enchantments, Harry stumbles into a sinister adventure when he finds a threeheaded dog guarding a room on the third floor. Then he hears of a missing stone with astonishing powers which could be valuable, dangerous, or both.", 'author': 'J.K. Rowling', 'keywords': '', 'moddate': '2012-07-01T14:20:39+10:00', 'title': 'Harry Potter and the Philosopher’s Stone', 'source': 'book1.pdf', 'total_pages': 228, 'page':

In [17]:
# Similairt search with scores 

docs_with_scores = vectorstore.similarity_search_with_score(query=query,
                                                k=3)

for doc, score in docs_with_scores:
    print("Score : ", score)
    print(doc.page_content[0:100])

Score :  0.5271847
They just looked at him, sm iling. And slowly, Harry looked 
into the faces of the other people in t
Score :  0.556026
knees, black hair and bright-green eyes. He wore round glasses 
held together with a lot of Se llota
Score :  0.5749573
Harry would have screamed, but he couldn’t make a sound. 
Where there should have been a b ack to Qu


In [18]:
# If we are implementing a chain we ll need a "Retriever"

retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs = {"k" : 3})

retrieved_docs = retriever.invoke("What was lord Volermort's alias ?")

for index, i in enumerate(retrieved_docs):
    print("Index : ",index+1)
    print(i.page_content[:100])
    print(i.metadata)

Index :  1
up at Ron and Hermione. 
‘I’ve found him!’ he whispered. ‘I’ve found Flamel! I told you I’d 
read th
{'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2012-05-01T23:57:27+10:00', 'subject': "When a letter arrives for unhappy but ordinary Harry Potter, a decade-old secret is revealed to him. His parents were wizards, killed by a Dark Lord's curse when Harry was just a baby, and which he somehow survived. Escaping from his unbearable Muggle guardians to Hogwarts, a wizarding school brimming with ghosts and enchantments, Harry stumbles into a sinister adventure when he finds a threeheaded dog guarding a room on the third floor. Then he hears of a missing stone with astonishing powers which could be valuable, dangerous, or both.", 'author': 'J.K. Rowling', 'keywords': '', 'moddate': '2012-07-01T14:20:39+10:00', 'title': 'Harry Potter and the Philosopher’s Stone', 'source': 'book1.pdf', 'total_pages': 228, 'page': 161, 'page

PDF -> Document (Pages) -> Chunking -> Embeddings -> FAISS Vector Store -> Similarity Search -> Relevant Chunks retrieved

# Adding LLM on top for retrieval

In [19]:
from langchain_huggingface import HuggingFacePipeline 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [24]:
model_name = "Qwen/Qwen3-0.6B" 

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="models")
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="cuda",
                                             cache_dir="models",
                                             dtype=torch.bfloat16)

pipe = pipeline("text-generation",
                model=model,
                tokenizer = tokenizer,
                max_new_tokens = 32,
                temperature = 0.6)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda


In [25]:
retriever = vectorstore.as_retriever(search_type = "similarity",
                                     search_kwargs = {"k" : 3}
                                     )

In [26]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       retriever = retriever,
                                       chain_type="stuff", #Simplest and direct method -> "stuffs" all retrieved documents into the context of large single prompt along with user questions which is then passed to the llm
                                       return_source_documents=True)

In [27]:
query = "Which house was harry sorted into ?"

result = qa_chain.invoke({"query" : query})

print(f"Answer  is \n{result['result']}")

print("\nSources")
for doc in result["source_documents"]:
    print(doc.metadata)

Answer  is 
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The Sorting Hat  91 
 
Slytherin, not Slytherin.’ 
‘Not Slytherin, eh?’ said the small voice. ‘Are you sure? You 
could be great, you know, it’s all here in your head, and Slytherin 
will help you on the way to greatn ess, no doubt about that – no? 
Well, if you’re sure – better be GRYFFINDOR!’ 
Harry heard the hat shout the last  word to the whole Hall. He 
took off the hat and walked shakily  towards the Gryffindor table. 
He was so relieved to have been chosen and not put in Slytherin, 
he hardly noticed that he was ge tting the loudest cheer yet. Percy

‘Am I?’ said Harry, feeling dazed. 
‘Goodness, didn’t you know, I’d have found out everything I 
could if it was me,’ said Hermio ne. ‘Do either of you know what 
house you’ll be in? I’ve been asking around and I hope I’m in 
Gryffindor, it sounds by far the b